# PID Controller Visualizer
Basic PID implementation in Python to demonstrate how the P, I, and D individually affect the performance of the controller.
<br/>
<br/>
Author: Morgan Visnesky 
<br/>
Date: 01-01-2021

In [ ]:
# PID CLASS

class PID:

  # Constructor
  def __init__(self, p, i, d, timeArray, goal):
    self.Kp = p 
    self.Ki = i 
    self.Kd = d 
    self.currentTime = 0
    self.previousTime = 0
    self.position = 0
    self.totalError = 0
    self.lastError = 0
    self.GOAL = goal
    self.timeSteps = len(timeArray)
    self.timeVals = timeArray # time values in seconds
    self.PIDvals = [] # holds values from PID 
    self.constVal = [] # holds constant value that PID is attempting to reach 

  # Compute's PID calculation
  def computePID(self, iteration):

      self.constVal.append(self.GOAL)

      self.currentTime = self.timeVals[iteration]*10
      if iteration != 0:
        self.previousTime = self.timeVals[iteration-1]*10
      else:
        self.previousTime = 0
      elapsedTime = (self.currentTime - self.previousTime)

      error = self.GOAL - self.position
      self.totalError += error * elapsedTime

      if elapsedTime != 0:
        derivError = (error-self.lastError) // elapsedTime
      else:
        derivError = 0

      posError = (self.Kp * error) + (self.Ki * self.totalError) + (self.Kd * derivError) # PID equation
      self.lastError = error
      self.position = self.position + posError
      self.PIDvals.append(self.position) # appends the calculated PID value for this time step to the array
      self.previousTime = self.currentTime
    
      x = iteration
      y = self.position
      return ((x,y))

# Test Plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

GOAL = 20 # constant value the controller will try and navigate to.
time_steps = 2000
t = np.arange(0, time_steps, 1) # time values in seconds

Kp = 0.202 # Proportional component of PID
Ki = 0.00015 # Integral component of PID
Kd = 0.207 # Derivitive component of PID
PID = PID(Kp, Ki, Kd, t, GOAL)



# For loop fills PIDvals array with PID controller values that correspond to each time
# step in the t array.
for i in range(time_steps):
  PID.computePID(i)
  
fig, ax = plt.subplots()
figure = matplotlib.pyplot.gcf() # gcf() = get current feature
figure.set_size_inches(16.5, 8.5) # resizes plot

ax.plot(t, PID.constVal, c='b',linewidth=6) # plots constant values
ax.plot(t, PID.PIDvals,c='r', linewidth=1) # plots PID values

# Sets labels for plot
ax.set(xlabel='Time (s)', ylabel='Speed (m/s)', title='PID Controller Visualization')

ax.grid() # Adds grid lines to plot
plt.rcParams.update({'font.size': 30}) # Changes font size of plot labels

#fig.savefig("PID_test.png") 
plt.show() # Displays plot

# PID Animation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import animation, rc
from IPython.display import HTML

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
figure = matplotlib.pyplot.gcf() # gcf() = get current feature
figure.set_size_inches(16.5, 4.5) # resizes plot
plt.close()

ax.t = np.arange(0, time_steps, 1)
ax.set_xlim(( 0, 2000))
ax.set_ylim((0, 30))

line, = ax.plot([], [], 'o')
line2, = ax.plot([], [], lw=2) 
PID = PID(0.202, 0.00015, 0.807, ax.t, 20)

# initialization function: plot the background of each frame
def init():
    line.set_data([], []) 
    line2.set_data(ax.t, np.full(2000,20))
    return (line2,)

# animation function. This is called sequentially  
def animate(i):
    vals = PID.computePID(i)
    line.set_data(vals[0], vals[1])
    return (line,)

ax.set(xlabel='Time (s)', ylabel='Speed (m/s)', title='PID Controller Visualization')
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=250, interval=100, blit=True)

# Note: below is the part which makes it work on Colab
rc('animation', html='jshtml')
anim